In [1]:
import pandas as pd
import numpy as np

In [2]:
all_dataset = pd.read_csv("data/arrhythmia.data", header=None)

##### choose columns to get from dataframe:

In [3]:
chosen_column = [0] + [i for i in range(3, 15)] + [all_dataset.columns[-1]]
max_trash = 20

##### ilter data and remove invalide symbols '?'

In [4]:
def clean_data(data):
    for col in data.columns:
        trash_row = (data[col] == "?")
        good_row = (data[col] != "?")
        if sum(trash_row) < max_trash:
            data = data[good_row]
        else:
            aver =  pd.to_numeric(data[good_row][col]).mean()
            data.loc[trash_row, col] = aver
        data[col] = pd.to_numeric(data[col])
        data[col] -= data[col].min()
    return data

##### instruction how to use default sklearn tools to split samples

In [5]:
def split_train_test(df):
    margin = int(0.8 * len(df))
    df = df.sample(frac=1)
    train = df.iloc[:margin]
    test = df.iloc[margin:]
    return train, test

In [6]:
#2,3 -> 2; 4,5 -> 3; 8, 9 -> 4; other -> 5
def reduce_class_6(y):
    y[(y == 2) | (y == 3)] = 2
    y[(y == 4) | (y == 5)] = 3
    y[(y == 8) | (y == 9)] = 4
    y[y > 5] = 5
    return y    

In [7]:
# 0 -> 0, other -> 1
def reduce_class_2(y):
    y[y > 0] = 1
    return y    

In [8]:
def from_data_to_xy(data):
    x = np.array(data[data.columns[:-1]], dtype=float)
    y = np.array(data[data.columns[-1]], dtype=float)
    return x, y

In [9]:
def from_data_to_xy_reduce(data):
    x = np.array(data[data.columns[:-1]], dtype=float)
    y = np.array(data[data.columns[-1]], dtype=float)
    y = reduce_class_2(y)
    return x, y

### Naive Bayes classification

#### Num of classes = 13 (all existing), using all rows

|Method | test on all data | cross validation|
|---|---|---|
|GaussianNB | 0.658  | 0.5 |
|ComplementNB | 0.536 | 0.48 |
|MultinomialNB | 0.47 | 0.43 |

#### Num of classes = 13 (all existing), using all rows except № 14
|Method | test on all data | cross validation|
|---|---|---|
|GaussianNB | 0.6832  | 0.6|
|ComplementNB | 0.5226 | 0.49|
|MultinomialNB | 0.5  | 0.45|

### Conclusion for 13 classes:
#### * when I delete almost emply column № 13, results becomes better
#### * choosing between Bayes classifiers Gaussian works better for me

### Naive Bayes classification

#### Num of classes = 2 (normal / illness) and = 6 (united by meaing)

|Method | num of classes = 2| num of classes = 6|
|---|---|---|
|GaussianNB | 0.72  |  0.58 |
|ComplementNB | 0.628 | 0.48 |
|MultinomialNB | 0.625 | 0.47 |


### Conclusion:
#### * even binal classification was not so easy task as it was expected
#### * we don't have enough knowledge to recombine the classes successfully

In [10]:
from sklearn import naive_bayes
def bayes_classifier(which='gaussian'):
    classifier = {
        'gaussian' : naive_bayes.GaussianNB(),
        'complement' : naive_bayes.ComplementNB(),
        'multinomial' : naive_bayes.MultinomialNB()
    }
    clf = classifier[which]
    return clf    

In [11]:
def check_accuracy(data, classifier, num_iter=1000, d_to_xy=from_data_to_xy_reduce):
#def check_accuracy(data, classifier, num_iter=1000, d_to_xy=from_data_to_xy):
    accuracy = []
    for i in range(num_iter):
        train_data, test_data = split_train_test(data)
        train_x, train_y = d_to_xy(train_data)
        test_x, test_y = d_to_xy(test_data)
        clf = classifier.fit(train_x, train_y)
        accuracy.append(sum(clf.predict(test_x) == test_y)  / test_y.shape[0])
    return accuracy

In [12]:
dataset = all_dataset[chosen_column]
clean_dataset = clean_data(dataset)

/home/ahavriushen/.local/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [ ]:
#which = 'multinomial'
#which = 'gaussian'
which = 'complement'

ac = check_accuracy(clean_dataset, bayes_classifier(which), 1000)
print(np.mean(ac))

### knn for 13 classes:

|num of nn| cv accuracy |
|---|---|
|1|     0.512808988764045|
|2|     0.5613483146067416|
|3|     0.5575280898876405|
|4|     0.5710112359550563|
|5|     0.579438202247191|
|6|     0.5697752808988763|
|7|     0.5767415730337079|
|8|     0.57|
|9|     0.5710112359550561|

#### Conclusion : n_neighbors = 5 gives higher result = 0.58

### knn for  6 classes:
|num of nn| cv accuracy |
|---|---|
|1|     0.5240449438202247|
|2|     0.5580898876404494|
|3|     0.5666292134831461|
|4|     0.5801123595505617|
|5|     0.5835955056179776|
|6|     0.582808988764045|
|7|     0.5807865168539326|
|8|     0.5896629213483147|
|9|     0.5776404494382023|
|10|     0.5802247191011236|
|11|     0.5692134831460673|
|12|     0.5671910112359551|

### knn for 2 classes:
|num of nn| cv accuracy |
|---|---|
|1|     0.6507865168539325|
|2|     0.6473033707865169|
|3|     0.6506741573033709|
|4|     0.656741573033708|
|5|     0.6574157303370788|
|6|     0.662808988764045|
|7|     0.6735955056179777|
|8|     0.6573033707865169|
|9|     0.6708988764044943|
|10|     0.6543820224719101|
|11|     0.6711235955056182|
|12|     0.650561797752809|


### Conclusion : 
#### * for 6 classes n_neighbors = 8 gives higher result = 0.59
#### * for 2 classes n_neighbors = 7 gives higher result = 0.673

In [24]:
from sklearn.neighbors import KNeighborsClassifier
for i in range(1,13):
    knn_classifier = KNeighborsClassifier(n_neighbors=i)
    ac = check_accuracy(clean_dataset, knn_classifier, 100)
    print(i, "   ", np.mean(ac))

7     0.6646067415730339
8     0.6573033707865169
9     0.6708988764044943
10     0.6543820224719101
11     0.6711235955056182
12     0.650561797752809


#### SVC for 13 classes => best result for linear kernel

|‘linear’| ‘poly’| ‘rbf’| ‘sigmoid’|
|---|---|---|---|
| 0.582 | 0.5346 | 0.5514 | 0.5437|

#### Also I tried to change number of classes and hyper parameter C
* svc linear for 6 classes: accuracy = 0.56
* svc linear for 2 classes with different C:

|C = 1 | 0.6787|
|---|---|
|C = 0.7 | 0.66 |
| C = 1.3 | 0.67 |

In [16]:
from sklearn.svm import SVC
clf = SVC(kernel='linear', C=1.3)
ac = check_accuracy(clean_dataset, clf, 10)
print(np.mean(ac))

0.6786516853932584


In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='poly', gamma='auto')
ac = check_accuracy(clean_dataset, clf, 10)
print(np.mean(ac))

In [25]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf', gamma='auto')
ac = check_accuracy(clean_dataset, clf, 10)
print(np.mean(ac))

0.5842696629213483


In [26]:
from sklearn.svm import SVC
clf = SVC(kernel='sigmoid', gamma='auto')
ac = check_accuracy(clean_dataset, clf, 10)
print(np.mean(ac))

0.449438202247191


In [13]:
chosen_factor_column = [0, 1] + [i for i in range(3, 15)] + [all_dataset.columns[-1]]

In [14]:
new_dataset = all_dataset[chosen_factor_column]
clean_new_dataset = clean_data(new_dataset)

### The second part: add factor variable Sex to database
Because sex has binar variable (0/1), I split the dataset according to the value of sex 

As we can see, here is very unbalanced dataset:
* amount of people with sex = 1, is 244
* amount of people with sex = 0, is 198


In [17]:
print('amount of people with sex = 1, is', sum(clean_new_dataset[1] > 0.5))
print('amount of people with sex = 0, is', sum(clean_new_dataset[1] < 0.5))

amount of people with sex = 1, is 244
amount of people with sex = 0, is 198


In [18]:
data1 = clean_new_dataset[clean_new_dataset[1] > 0.5][chosen_column]
data2 = clean_new_dataset[clean_new_dataset[1] < 0.5][chosen_column]

### For people with sex = 1, byes classifications gave folloving accuracy:
|distribution| with all columns| without 13 columns |
|---|---|---|
|'multinomial' | 0.4779183673469388 | 0.46075510204081627|
|'gaussian' | 0.5273061224489797 | 0.6748367346938776 |
|'complement' | 0.5630816326530612 | 0.5624489795918367 |

#### As before, gaussian is the better choise with mean accuracy of 0.67

In [57]:
ac = check_accuracy(data1, bayes_classifier('multinomial'), 1000)
print(np.mean(ac))

ac = check_accuracy(data1, bayes_classifier('gaussian'), 1000)
print(np.mean(ac))

ac = check_accuracy(data1, bayes_classifier('complement'), 1000)
print(np.mean(ac))

0.4659795918367347


### For people with sex = 0, byes classifications gave folloving accuracy:
|distribution| with all columns| without 13 columns |
|---|---|---|
|'multinomial' | 0.354375 | 0.336775|
|'gaussian' | 0.305225 |  0.42795 |
|'complement' |  0.340825 | 0.33185 |

#### As before, gaussian is the better choise with mean accuracy of 0.42
#### Reasone of much worse result is 

In [ ]:
ac = check_accuracy(data2, bayes_classifier('multinomial'), 1000)
print(np.mean(ac))

ac = check_accuracy(data2, bayes_classifier('gaussian'), 1000)
print(np.mean(ac))

ac = check_accuracy(data2, bayes_classifier('complement'), 1000)
print(np.mean(ac))

data1 knn


### For people with sex = 1, knn gave folloving accuracy:

|num of nn| cv accuracy |
|---|---|
|5|     0.6587142857142857|
|6|     0.6606938775510204|
|7|     0.6598163265306123|
|8|     0.6574285714285714|
|9|     0.6592244897959183|
|10|     0.6541428571428571|

Conclusion : n_neighbors = 6 gives higher result = 0.66 and it's better accuracy then for all data before splitting


### For people with sex = 1, knn gave folloving quite poor accuracy:
|num of nn| cv accuracy |
|---|---|
|5|     0.44755|
|6|     0.443775|
|7|     0.441775|

In [40]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=13)

for i in range(1,14):
    knn_classifier = KNeighborsClassifier(n_neighbors=i)
    ac = check_accuracy(data1, knn_classifier, 1000)
    print(i, "   ", np.mean(ac))

5     0.6587142857142857
6     0.6606938775510204
7     0.6598163265306123
8     0.6574285714285714
9     0.6592244897959183
10     0.6541428571428571
11     0.6573265306122449
12     0.6537142857142857
13     0.6521224489795918


In [41]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=13)

for i in range(5,14):
    knn_classifier = KNeighborsClassifier(n_neighbors=i)
    ac = check_accuracy(data2, knn_classifier, 1000)
    print(i, "   ", np.mean(ac))

5     0.44755
6     0.443775
7     0.441775
8     0.43277499999999997
9     0.43129999999999996
10     0.42892499999999995
11     0.43079999999999996
12     0.42047500000000004
13     0.4161


   ### SVC for people with sex = 1
|‘linear’| ‘poly’| ‘rbf’| ‘sigmoid’|
|---|---|---|---|
| 0.682 | 0.614 | 0.6516 | 0.66|
    
   ### SVC for people with sex = 0 
|‘linear’| ‘poly’| ‘rbf’| ‘sigmoid’|
|---|---|---|---|
| 0.461 | 0.379 | 0.4197 | 0.4225|


### General Conclusion :
SVC with linear kernel was the most suitable for classification for all combinations of datasets and class labels

In [46]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
ac = check_accuracy(data1, clf, 100)
print(np.mean(ac))

0.6828571428571426


In [47]:
from sklearn.svm import SVC
clf = SVC(kernel='poly', gamma='auto')
ac = check_accuracy(data1, clf, 100)
print(np.mean(ac))

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

0.6140816326530613


In [48]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf', gamma='auto')
ac = check_accuracy(data1, clf, 100)
print(np.mean(ac))

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

0.6516326530612245


/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

In [49]:
from sklearn.svm import SVC
clf = SVC(kernel='sigmoid', gamma='auto')
ac = check_accuracy(data1, clf, 100)
print(np.mean(ac))

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

0.6606122448979592


/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

In [50]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
ac = check_accuracy(data2, clf, 100)
print(np.mean(ac))

0.4617499999999999


In [51]:
from sklearn.svm import SVC
clf = SVC(kernel='poly')
ac = check_accuracy(data2, clf, 100)
print(np.mean(ac))

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

0.379


In [52]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf')
ac = check_accuracy(data2, clf, 100)
print(np.mean(ac))

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

0.41975


/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

In [53]:
from sklearn.svm import SVC
clf = SVC(kernel='sigmoid')
ac = check_accuracy(data2, clf, 100)
print(np.mean(ac))

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

0.4225


/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

### Classes distribution: 

In [148]:
import collections

_, y_ = from_data_to_xy(clean_dataset)

print(collections.Counter(y))

print(np.unique(y))

Counter({0.0: 243, 9.0: 50, 1.0: 39, 5.0: 25, 15.0: 21, 2.0: 15, 3.0: 14, 4.0: 13, 8.0: 9, 14.0: 5, 13.0: 4, 7.0: 2, 6.0: 2})
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 13. 14. 15.]
